In [1]:
# !pip install pandas	
# !pip install numpy
# !pip install tqdm	
# !pip install matplotlib

In [1]:
# 라이브러리
import random
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from numpy.random import randint, rand

In [2]:
# 데이터 불러오기
machine_info = pd.read_csv("./data/machine_info.csv")
order_info = pd.read_csv("./data/order_info.csv")
order_info_nl = pd.read_csv("./data/order_info_nl.csv", header=None)
order_info_nl_eng = pd.read_csv("./data/order_info_nl_eng.csv")

In [3]:
order_info_nl.columns = ['영업납기', '중산도면', '단가', '수량', '선급', '자연어주문']

In [4]:
# 전처리 (1): column name 변경
order_info = order_info.rename(columns={
    '영업납기': 'time',
    '중산도면': 'item',
    '단가': 'cost',
    '수량': 'qty',
    '선급': 'urgent'
})

machine_info = machine_info.rename(columns={
    'JSDWG': 'item',
    'MCNO': 'machine',
    'AVG_CT': 'capacity'
})

order_info_nl = order_info_nl.rename(columns={
    '영업납기': 'time',
    '중산도면': 'item',
    '단가': 'cost',
    '수량': 'qty',
    '선급': 'urgent',
    '자연어주문': 'nl_order'
})

order_info_nl_eng.columns = ['nl_order']

In [5]:
# 머신인포 dropna, 4자리 숫자 문자열로 변환
machine_info.dropna(inplace=True)
machine_info['machine'] = machine_info['machine'].apply(lambda x: str(int(x)).zfill(4))

In [6]:
# 아이템 -> item01
# qty -> 그대로
# 날짜 -> 일수(날짜 - 2021-01-01) date001(1~365) / 자연어는 수정X
# 선급 -> 일단 자연어 그대로
# cost -> 일단 그대로
# machine -> machine01
# capacity -> 일단 그대로

In [7]:
# 아이템 -> item01
unique_items = sorted(order_info['item'].unique(), reverse=True)

items_mapping = {
        item: f"item{str(i+1).zfill(2)}"
        for i, item in enumerate(unique_items)
    }

def change_item_name(target):
    result = str(target)
    for k, v in items_mapping.items():
        result = result.replace(k, v)

    return result

change_item_name('057791')

'item87'

In [8]:
machine_info['item'] = machine_info['item'].apply(change_item_name)
cond = machine_info['item'].isin(items_mapping.values())
machine_info[cond].head()

,item,machine,capacity
897,item96,0408,4.89
898,item96,0409,5.92
899,item96,0410,4.91
900,item96,0412,3.50
901,item96,0416,1.03


In [9]:
order_info['item'] = order_info['item'].apply(change_item_name)
order_info.head()

,time,item,cost,qty,urgent
0,2021-05-13,item15,25870,318,검사품
1,2021-05-24,item16,16229,383,검사품
2,2021-05-30,item96,8333,19,NaN
3,2021-06-03,item91,36533,4,NaN
4,2021-06-18,item87,45500,196,검사품


In [10]:
order_info_nl['item'] = order_info_nl['item'].apply(change_item_name)
order_info_nl['nl_order'] = order_info_nl['nl_order'].apply(change_item_name)
order_info_nl.head()

,time,item,cost,qty,urgent,nl_order
0,2021-05-13,item15,25870,318,검사품,item15 맞지? 318개 하는거? 21년 5월 13일까지 하면 되구 단가가 25...
1,2021-05-24,item16,16229,383,검사품,헐 방금 item16 도면 왔는데 21년 5월 24일까지래. 383개 해야되구 16...
2,2021-05-30,item96,8333,19,NaN,"저기 item96 도면 19개 해야돼. 21년 5월 30일까지구 8,333원. 그리..."
3,2021-06-03,item91,36533,4,NaN,어우 빨리 봐봐. item91 4개 21년 6월 3일까지 해야되는데 어떡하지? 단가...
4,2021-06-18,item87,45500,196,검사품,item87 맞지? 196개 하는거? 21년 6월 18일까지 하면 되구 단가가 45...


In [11]:
order_info_nl_eng['nl_order'] = order_info_nl_eng['nl_order'].apply(change_item_name)
order_info_nl_eng.head()

,nl_order
0,"item15, right? 318? You can do it by May 13th,..."
1,"Oh, I just got the item16 drawing, and it says..."
2,"There, I need to do 19 drawings for item96. It..."
3,"Hey, look quickly. item91 4 I have to do it by..."
4,"item87, right? Doing 196? You can do it until ..."


In [12]:
# 날짜 -> 일수(날짜 - 2021-01-01) (1~365)
def change_time_name(target):
    time = pd.to_datetime(target)
    base_date = pd.to_datetime('2021-01-01')

    return f'date{str((time-base_date).days+1).zfill(3)}'

change_time_name('2021-01-01'), change_time_name('2021-12-31')

('date001', 'date365')

In [13]:
order_info['time'] = order_info['time'].apply(change_time_name)
order_info.head()

,time,item,cost,qty,urgent
0,date133,item15,25870,318,검사품
1,date144,item16,16229,383,검사품
2,date150,item96,8333,19,NaN
3,date154,item91,36533,4,NaN
4,date169,item87,45500,196,검사품


In [14]:
order_info_nl['time'] = order_info_nl['time'].apply(change_time_name)
order_info_nl.head()

,time,item,cost,qty,urgent,nl_order
0,date133,item15,25870,318,검사품,item15 맞지? 318개 하는거? 21년 5월 13일까지 하면 되구 단가가 25...
1,date144,item16,16229,383,검사품,헐 방금 item16 도면 왔는데 21년 5월 24일까지래. 383개 해야되구 16...
2,date150,item96,8333,19,NaN,"저기 item96 도면 19개 해야돼. 21년 5월 30일까지구 8,333원. 그리..."
3,date154,item91,36533,4,NaN,어우 빨리 봐봐. item91 4개 21년 6월 3일까지 해야되는데 어떡하지? 단가...
4,date169,item87,45500,196,검사품,item87 맞지? 196개 하는거? 21년 6월 18일까지 하면 되구 단가가 45...


In [15]:
# machine -> machine01
unique_machines = sorted(machine_info['machine'].dropna().unique())

machines_mapping = {
        machine: f"machine{str(i+1).zfill(2)}"
        for i, machine in enumerate(unique_machines)
    }

def change_machine_name(target):
    result = str(target)
    for k, v in machines_mapping.items():
        result = result.replace(k, v)

    return result

change_machine_name('0421')

'machine19'

In [16]:
machine_info['machine'] = machine_info['machine'].apply(change_machine_name)
machine_info.head()

,item,machine,capacity
0,050060,machine28,1.08
1,050093,machine04,7.13
2,050093,machine08,4.67
3,050093,machine10,4.50
4,050093,machine14,3.92


In [17]:
# 전처리 - order
def preprocess_order(df):
    # 긴급 생산 요건이 있는 경우 1, 그렇지 않은 경우는 0
    df['urgent'] = df['urgent'].fillna(0)

    for i in range(len(df)):
        if df.loc[i, 'urgent'] != 0:
            df.loc[i, 'urgent'] = 1

    df.dropna(axis=0, inplace=True)

    # 같은 종류의 order 묶기
    df = df.groupby(['time','item','cost','urgent']).sum().reset_index()
    
    return df

# 전처리 - machine
def preprocess_machine(df):
    df.dropna(axis=0, inplace=True)

    return df

In [18]:
# 슬라이딩 윈도우 함수 정의
def create_shifted_dataset(df, window_size=5):
    datasets = []
    for i in range(len(df) - window_size + 1):
        window = df.iloc[i:i+window_size].reset_index(drop=True)
        datasets.append(window)
    return datasets

In [24]:
machine_info = preprocess_machine(machine_info)

def make_dataset(order_info_nl, machine_info, window_size=5):
    dataset = create_shifted_dataset(order_info_nl, window_size=window_size)
    dataset_for_ga = list()
    dataset_for_nl = list()
    dataset_for_order = list()

    for data in dataset:
        order = data.iloc[:, :5]
        nl = data.iloc[:, 5:]

        order = preprocess_order(order)
        dataset_for_order.append(order)
        order = pd.merge(order, machine_info, on='item', how='inner')

        dataset_for_ga.append(order)
        dataset_for_nl.append(nl)

    return dataset_for_ga, dataset_for_nl, dataset_for_order

# raw_order_info_nl_eng = order_info_nl_eng.copy()
order_info_nl_eng = pd.concat([order_info, order_info_nl_eng], axis=1)
dataset_for_ga, dataset_for_nl, dataset_for_order = make_dataset(order_info_nl_eng, machine_info)

In [26]:
# 노트북 셀 (dataset_for_ga가 생성된 이후)

if 'dataset_for_ga' in globals() and dataset_for_ga:
    rng = np.random.RandomState(seed=20250526) # 시드 고정 (결과 비교를 위해)
    # TODO: random_ls를 만들 때 dataset_for_ga의 실제 길이로 size를 제한해야 할 수 있음
    #       (만약 dataset_for_ga 길이가 10보다 작다면 에러 발생 가능)
    #       여기서는 dataset_for_ga 길이가 충분히 크다고 가정.
    #       또는, 테스트를 위해 실제 샘플 개수를 줄여도 좋음 (예: size=1 또는 2)
    num_samples_to_test = min(10, len(dataset_for_ga)) # 실제 샘플 수 제한
    random_ls = rng.choice(a=len(dataset_for_ga), size=num_samples_to_test, replace=False)
    temp0 = [dataset_for_ga[i] for i in random_ls]
    print(f"테스트용 데이터 샘플 {len(temp0)}개 준비 완료 (temp0).")
else:
    print("오류: dataset_for_ga가 준비되지 않았습니다.")
    temp0 = []

테스트용 데이터 샘플 10개 준비 완료 (temp0).


In [27]:
def ga_preprocess_sorted(dataset):
    T = sorted(list(set(dataset['time'])))
    I = sorted(list(set(dataset['item'])))
    J = sorted(list(set(dataset['machine'])))

    # cit = 날짜 t에, item i가 미생산 될 때 발생하는 비용
    cit = dict()
    for i in I:
        for t in T:
            temp_dataset = dataset[(dataset['item'] == i) & (dataset['time'] == t)]
            if len(temp_dataset) != 0:
                value = list(set(temp_dataset['cost']))[0]
                cit[i, t] = value
            else:
                cit[i, t] = 0

    # pit = 날짜 t에, item i가 긴급 생산이 필요한 경우 1, 그렇지 않으면 0
    pit = dict()
    for i in I:
        for t in T:
            temp_dataset = dataset[(dataset['item'] == i) & (dataset['time'] == t)]
            if len(temp_dataset) != 0:
                value = list(set(temp_dataset['urgent']))[0]
                pit[i, t] = value
            else:
                pit[i, t] = 0

    # dit = 날짜 t에 item i마다 생산되어야 하는 요구량
    dit = dict()
    for i in I:
        for t in T:
            temp_dataset = dataset[(dataset['item'] == i) & (dataset['time'] == t)]
            if len(temp_dataset) != 0:
                value = list(set(temp_dataset['qty']))[0]
                dit[i, t] = value
            else:
                dit[i, t] = 0

    # mijt= 날짜 t에 NC machine j가 item i를 생산할 수 있는 능력
    mijt = dict()
    for i in I:
        for j in J:
            temp_dataset = dataset[
                (dataset['item'] == i) &
                (dataset['machine'] == j)
            ]

            if len(temp_dataset) != 0:
                value = list(set(temp_dataset['capacity']))[0]
                for t in T:
                    mijt[i, j, t] = value
            else:
                for t in T:
                    mijt[i, j, t] = 0

    return T, I, J, cit, pit, dit, mijt


In [28]:
def ga_preprocess(dataset):
    T = list(set(dataset['time']))
    I = list(set(dataset['item']))
    J = list(set(dataset['machine']))

    # cit = 날짜 t에, item i가 미생산 될 때 발생하는 비용
    cit = dict()
    for i in I:
        for t in T:
            temp_dataset = dataset[(dataset['item'] == i) & (dataset['time'] == t)]
            if len(temp_dataset) != 0:
                value = list(set(temp_dataset['cost']))[0]
                cit[i, t] = value
            else:
                cit[i, t] = 0

    # pit = 날짜 t에, item i가 긴급 생산이 필요한 경우 1, 그렇지 않으면 0
    pit = dict()
    for i in I:
        for t in T:
            temp_dataset = dataset[(dataset['item'] == i) & (dataset['time'] == t)]
            if len(temp_dataset) != 0:
                value = list(set(temp_dataset['urgent']))[0]
                pit[i, t] = value
            else:
                pit[i, t] = 0

    # dit = 날짜 t에 item i마다 생산되어야 하는 요구량
    dit = dict()
    for i in I:
        for t in T:
            temp_dataset = dataset[(dataset['item'] == i) & (dataset['time'] == t)]
            if len(temp_dataset) != 0:
                value = list(set(temp_dataset['qty']))[0]
                dit[i, t] = value
            else:
                dit[i, t] = 0

    # mijt= 날짜 t에 NC machine j가 item i를 생산할 수 있는 능력
    mijt = dict()
    for i in I:
        for j in J:
            temp_dataset = dataset[
                (dataset['item'] == i) &
                (dataset['machine'] == j)
            ]

            if len(temp_dataset) != 0:
                value = list(set(temp_dataset['capacity']))[0]
                for t in T:
                    mijt[i, j, t] = value
            else:
                for t in T:
                    mijt[i, j, t] = 0

    return T, I, J, cit, pit, dit, mijt

# 머신 j가 시간 t에 아이템 i를 만들수 없을 경우 0으로 처리하기 위함함
def decode(mijt, xijt):
    for j in J:
        for t in T:
            for i in I:
                if mijt[i, j, t] == 0:
                    xijt[i, j, t] = 0
    return xijt

# 목적함수를 최적화하기 위한 xijt 조합을 임의로 생성하는 함수
def generation_xijt():
    xijt = {}

    for i in I:
        for j in J:
            for t in T:
                if dit[i, t] > 0:
                    xijt[i, j, t] = random.uniform(0, 1)
                else:
                    xijt[i, j, t] = 0

    xijt = decode(mijt, xijt)
    return xijt

def dict2list(xijt):
    return list(xijt.values())

def list2dict(bitstring, type='xijt'):
    if type == 'xijt':
        _keys = xijt_keys
    elif type == 'mijt':
        _keys = mijt_keys

    xijt = {}
    for idx, value in enumerate(bitstring):
        xijt[_keys[idx]] = value

    return xijt

# 제약조건 확인
# 기계 하루 총 생산시간은 600분
# j기계로 600분 안에 아이템 i를 생산할 수 없다면 아이템 생산하지 않도록 하기 위함.
def constraint_check(xijt):
    for j in J:
        for t in T:
            check_value = sum(mijt[i, j, t] * round(xijt[i, j, t] * dit[i, t]) for i in I) <= 600

            if check_value is False:
                for i in I:
                    xijt[i, j, t] = 0

    return xijt

# 목적 함수
def objective(xijt):
    uit = {}
    xijt = constraint_check(xijt)

    for i in I:
        for t in T:
            u = dit[i, t] - sum(round(xijt[i, j, t] * dit[i, t]) for j in J)
            if u >= 0:
                uit[i, t] = u
            else:
                uit[i, t] = abs(u) * 10_000_000

    objective = sum(uit[i, t] * cit[i, t] * pit[i, t] for i in I for t in T)
    return objective

# Tournament selection
def selection(pop, scores, n_pop):
    # n_pop은 한 세대를 구성하는 염색체 수이며, 하이퍼파라미터 값
    k = round(n_pop * 0.5)
    selection_ix = randint(len(pop))
    
    for ix in randint(0, len(pop), k - 1):
        if scores[ix] < scores[selection_ix]:
            selection_ix = ix
            
    return pop[selection_ix]

def crossover(p1, p2, r_cross):
    p1 = dict2list(p1)
    p2 = dict2list(p2)
    c1, c2 = p1.copy(), p2.copy()

    if rand() < r_cross:
        base_bitstring = dict2list(mijt)

        # crossover (1): 유효 인덱스 추출
        valid_index = []
        for i in range(len(base_bitstring)):
            if base_bitstring[i] > 0:
                valid_index.append(i)

        n = round(r_cross * len(valid_index))
        pt_0 = random.sample(valid_index, n)
        pt_1 = list(set(valid_index) - set(pt_0))

        # crossover (2): 교차 적용
        for i in pt_0:
            c1[i] = p2[i]
            c2[i] = p1[i]

        for i in pt_1:
            c1[i] = p2[i]
            c2[i] = p1[i]

    return [c1, c2]

def mutation(bitstring, r_mut):
    for i in range(len(bitstring)):
        if rand() < r_mut:
            base_bitstring = dict2list(mijt)

            # mutation (1): 유효 인덱스 찾기
            valid_index = []
            for j in range(len(base_bitstring)):
                if base_bitstring[j] > 0:
                    valid_index.append(j)

            n = round(r_mut * len(valid_index))

            # mutation (2): 무작위 인덱스에서 값 변경
            pt_0 = random.sample(valid_index, n)
            for idx in pt_0:
                bitstring[idx] = random.uniform(0, 1)

def genetic_algorithm(bounds, n_iter, n_pop, r_cross, r_mut):
    # Utility
    log = []
    log_detail = []
    best_gen = 0

    # GA algorithm 초기화
    pop = [generation_xijt() for _ in range(n_pop)]
    best = decode(mijt, pop[0])
    best_eval = objective(best)
    print(best_eval)

    for gen in tqdm(range(n_iter)):
        # 현재 세대 해를 디코딩
        decoded = [decode(mijt, p) for p in pop]

        # 목적함수 값 계산
        scores = [objective(d) for d in decoded]

        # 가장 좋은 해 갱신
        for i in range(n_pop):
            if scores[i] < best_eval:
                best, best_eval = pop[i], scores[i]
                print(f'>best! {gen}, {scores[i]}')
                best_gen = gen
            else:
                # stop rule: 개선 없이 100세대 이상일 경우 중단
                if gen - best_gen > 200:
                    print('stop')
                    return [best, best_eval, log, log_detail]

        # 부모 해 집합 선택
        selected = [selection(pop, scores, n_pop) for _ in range(n_pop)]
        log_detail.append([gen, objective(selected[0])])

        # 자식 세대 생성
        children = []
        for i in range(0, n_pop, 2):
            p1, p2 = selected[i], selected[i + 1]
            for c in crossover(p1, p2, r_cross):
                mutation(c, r_mut)
                children.append(list2dict(c))

        pop = children
        log.append([gen, best_eval])

    return [best, best_eval, log, log_detail]

def set_hyper_parameters():
    # 하이퍼파라미터 설정
    hyper_parameters = pd.DataFrame({
        'index':    ['index_1', 'index_2', 'index_3', 'index_4',
                    'index_5', 'index_6', 'index_7', 'index_8'],
        'n_iter':   [500, 500, 500, 500, 500, 500, 500, 500],
        'n_pop':    [10, 20, 40, 20, 20, 20, 20, 20],
        'r_cross':  [0.4, 0.4, 0.4, 0.1, 0.2, 0.3, 0.4, 0.4],
        'r_mut':    [0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.1, 0.6]
    })

    # 결과 기록용 열 추가
    hyper_parameters['objective'] = np.nan
    hyper_parameters['time'] = np.nan

    return hyper_parameters

def find_hyper_parameters(hyper_parameters):
    log_list = []

    for i in range(len(hyper_parameters)):
        parameter = hyper_parameters.iloc[i]
        index_nm = parameter['index']
        print(f'{index_nm}')

        start = datetime.datetime.now()

        # 초기 해 및 키 정의
        xijt = generation_xijt()
        global xijt_keys, mijt_keys
        xijt_keys = list(xijt.keys())
        mijt_keys = list(mijt.keys())

        # 하이퍼파라미터 설정
        n_iter = parameter['n_iter']
        n_pop = parameter['n_pop']
        r_cross = parameter['r_cross']
        r_mut = parameter['r_mut']

        # 유전 알고리즘 실행
        best, score, log, log_detail = genetic_algorithm(mijt, n_iter, n_pop, r_cross, r_mut)

        # 실행 시간 및 결과 기록
        elapsed_time = datetime.datetime.now() - start
        hyper_parameters.loc[i, 'time'] = elapsed_time
        hyper_parameters.loc[i, 'objective'] = score

        log_list.append(log)

    # 결과 출력
    return hyper_parameters, log_list


In [29]:
rng = np.random.RandomState(seed=20250526)
random_ls = rng.choice(a=len(dataset_for_ga), size=(10, ))
random_ls

array([ 68,   6,  77,  91,  29,  10,  58, 109,   0, 111])

In [30]:
import pickle

# 적합한 하이퍼 파라미터 찾는 과정
temp0 = [dataset_for_ga[i] for i in random_ls]

hyper_parameters_ls, log_list_ls = list(), list()

# 유전 알고리즘
def ga_get_gt(dataset_for_ga):
    for data in dataset_for_ga:
        global T, I, J, cit, pit, dit, mijt
        T, I, J, cit, pit, dit, mijt = ga_preprocess(data)

        hyper_parameters = set_hyper_parameters()
        hyper_parameters, log_list = find_hyper_parameters(hyper_parameters)

        hyper_parameters_ls.append(hyper_parameters)
        log_list_ls.append(log_list)
        with open('hyper_parameters_ls_n5.pkl', 'wb') as f:
            pickle.dump(hyper_parameters_ls, f)

    return hyper_parameters_ls, log_list_ls

hyper_parameters_ls, log_list_ls = ga_get_gt(temp0)

index_1
57215020209897


  3%|██▏                                                                           | 14/500 [00:00<00:10, 45.84it/s]

>best! 6, 56400020104557
>best! 8, 41250019841140


  8%|██████                                                                        | 39/500 [00:00<00:09, 48.40it/s]

>best! 29, 29975019554329


...

>best! 14, 222739993040000


  4%|██▊                                                                           | 18/500 [00:08<03:37,  2.21it/s]

>best! 18, 210471170969000


  4%|██▉                                                                           | 19/500 [00:08<03:45,  2.13it/s]


KeyboardInterrupt: 

In [50]:
hyper_parameters_df = hyper_parameters_ls[0][['index', 'n_iter', 'n_pop', 'r_cross', 'r_mut']]

for hyper_parameters in hyper_parameters_ls:
    temp = hyper_parameters[['objective']]
    hyper_parameters_df = pd.concat([hyper_parameters_df, temp], axis=1)

hyper_parameters_df.columns = ['index', 'n_iter', 'n_pop', 'r_cross', 'r_mut'] + list(random_ls)

hyper_parameters_df

,index,n_iter,n_pop,r_cross,r_mut,68,6,77,91,29,10,58,109,0,111
0,index_1,500,10,0.4,0.4,2.304075e+07,0.0,1548384.0,1.685775e+14,5.874482e+13,0.0,0.0,1.818114e+13,1315834.0,1.256976e+13
1,index_2,500,20,0.4,0.4,3.760016e+12,0.0,946276.0,1.649281e+14,4.933088e+13,0.0,0.0,1.548280e+13,542225.0,1.048470e+13
2,index_3,500,40,0.4,0.4,4.200021e+12,0.0,958140.0,1.383163e+14,4.979328e+13,0.0,0.0,1.285872e+13,966536.0,7.946400e+12
3,index_4,500,20,0.1,0.4,1.937002e+13,0.0,808560.0,1.686056e+14,5.815256e+13,0.0,0.0,1.544390e+13,1129548.0,1.242528e+13
4,index_5,500,20,0.2,0.4,1.410023e+12,0.0,633574.0,1.620072e+14,5.480808e+13,0.0,0.0,1.314768e+13,1415619.0,1.329216e+13
5,index_6,500,20,0.3,0.4,1.937638e+07,0.0,974566.0,1.498418e+14,5.327286e+13,0.0,0.0,9.883240e+12,1023926.0,1.612720e+13
6,index_7,500,20,0.4,0.1,2.369420e+07,0.0,280506.0,1.587894e+14,5.448986e+13,0.0,0.0,1.560384e+13,1435246.0,7.368481e+12
7,index_8,500,20,0.4,0.6,1.020002e+13,0.0,1146728.0,1.770980e+14,5.001406e+13,0.0,0.0,1.264402e+13,987182.0,1.030496e+13


In [31]:
import pickle

# 저장된 피클 불러오기기
# with open('log.pkl', 'rb') as f:
#     loaded_data = pickle.load(f)

# best_ls, score_ls, log_ls, log_detail_ls = loaded_data

# 데이터 설정정
# temp = [dataset_for_ga[i] for i in range(10)]

best_ls, score_ls, log_ls, log_detail_ls, gt_ls = list(), list(), list(), list(), list()

# 유전 알고리즘
def ga_get_gt(dataset_for_ga):
    data_len = len(dataset_for_ga)
    
    for i, data in enumerate(dataset_for_ga):
        global T, I, J, cit, pit, dit, mijt
        T, I, J, cit, pit, dit, mijt = ga_preprocess(data)

        # 입력 데이터 생성
        xijt = generation_xijt()
        global xijt_keys, mijt_keys
        xijt_keys = list(xijt.keys())
        mijt_keys = list(mijt.keys())

        # 유전 알고리즘 하이퍼파라미터 설정 - index 07 적용
        n_iter = 500       # 반복할 세대 수
        n_pop = 20         # 세대당 염색체 수
        r_cross = 0.4      # 교차율
        r_mut = 0.1        # 변이율

        # 유전 알고리즘 실행
        best, score, log, log_detail = genetic_algorithm(mijt, n_iter, n_pop, r_cross, r_mut)

        best_ls.append(best)
        score_ls.append(score)
        log_ls.append(log)
        log_detail_ls.append(log_detail)

        with open('log_n5.pkl', 'wb') as f:
            pickle.dump([best_ls, score_ls, log_ls, log_detail_ls], f)
        print(f'{i+1}번째 log 저장 완료. 총 {data_len}개')

        # 해 솔루션 저장용 딕셔너리 초기화
        solution_ = dict()

        # xijt 디코딩
        xijt = decode(mijt, xijt)

        # best 해를 기반으로 해를 계산
        for i in I:
            for j in J:
                for t in T:
                    solution_[(i, j, t)] = round(best[i, j, t] * dit[i, t])

        # 딕셔너리를 DataFrame으로 변환
        sol = pd.DataFrame.from_dict(solution_, orient='index').reset_index()
        sol.columns = ['(item, machine, time)', 'qty']

        gt_ls.append(sol[sol['qty'] > 0])

        with open('gt_n5_k010k.pkl', 'wb') as f:
            pickle.dump(gt_ls, f)

        # print('{}번째 gt 저장 완료. 총 {}개'.format(i + 1, data_len))

        # print('------------------------')
        # print()
    # return gt_ls

ga_get_gt(dataset_for_ga)

31159690513360


  0%|▎                                                                              | 2/500 [00:00<00:31, 15.95it/s]

>best! 0, 6653672
>best! 1, 6285361
>best! 2, 5082676
>best! 3, 4919956


  2%|█▎                                                                             | 8/500 [00:00<00:31, 15.84it/s]

>best! 5, 4442480
>best! 7, 2768916


...

>best! 12, 4266500


 10%|███████▊                                                                      | 50/500 [00:01<00:10, 42.14it/s]

>best! 43, 2016100


 49%|█████████████████████████████████████▌                                       | 244/500 [00:05<00:05, 42.67it/s]

stop
123번째 log 저장 완료. 총 123개


In [50]:
with open('gt_n5_k010k.pkl', 'rb') as f:
    gt_ls = pickle.load(f)
gt_ls[1]

,"(item, machine, time)",qty
23,"(item16, machine07, date144)",3
47,"(item16, machine10, date144)",136
67,"(item16, machine23, date144)",16
83,"(item16, machine30, date144)",53
87,"(item16, machine08, date144)",128
120,"(item87, machine05, date169)",30
140,"(item87, machine01, date169)",92
196,"(item87, machine21, date169)",74
233,"(item91, machine35, date154)",2
253,"(item91, machine09, date154)",3


In [33]:
# GT 결과값 데이테프레임화
for idx, gt in enumerate(gt_ls):
    gt[['item', 'machine', 'time']] = pd.DataFrame(gt['(item, machine, time)'].tolist(), index=gt.index)
    gt = gt.drop(columns='(item, machine, time)')
    gt = gt[['item', 'machine', 'time', 'qty']].sort_values(['time', 'item', 'machine', 'qty'])
    gt_ls[idx] = gt

gt_ls[0]

,item,machine,time,qty
608,item15,machine20,date133,133
598,item15,machine21,date133,57
553,item15,machine28,date133,52
558,item15,machine34,date133,46
91,item16,machine08,date144,88
46,item16,machine10,date144,164
66,item16,machine23,date144,32
86,item16,machine30,date144,69
465,item96,machine08,date150,9
410,item96,machine09,date150,16


In [110]:
# gt 전처리
# for idx, gt in enumerate(gt_ls):
#     gt['machine'] =   gt['machine'].apply(lambda x: str(int(x)).zfill(4))
#     gt['item'] =  gt['item'].apply(change_item_name)
#     gt['time'] =  gt['time'].apply(change_time_name)
#     gt['machine'] = gt['machine'].apply(change_machine_name)
#     gt_ls[idx] = gt

# gt_ls[0]

In [111]:
# order 정보 묶기
def combine_order_nl(data):
    combined_data = '\n'.join(f'• {text}' for text in data['nl_order'])

    return combined_data

combine_order_nl(dataset_for_nl[0])

"• item15, right? 318? You can do it by May 13th, 2021, and the unit price is 25,870 won. Oh, and the inspection product.\n• Oh, I just got the item16 drawing, and it says it's due on May 24th, 2021. I need to do 383 of them, and it costs 16,229 won, and it's an inspection item.\n• There, I need to do 19 drawings for item96. It's until May 30th, 2021. It's 8,333 won. And nothing special.\n• Hey, look quickly. item91 4 I have to do it by June 3rd, 2021. What should I do? The unit price is 36,533 won and there's nothing special about it.\n• item87, right? Doing 196? You can do it until June 18th, 2021, and the unit price is 45,500 won. Oh, and the inspection product"

In [112]:
order_nl_for_dataset = [combine_order_nl(data) for data in dataset_for_nl]
order_nl_for_dataset[0]

"• item15, right? 318? You can do it by May 13th, 2021, and the unit price is 25,870 won. Oh, and the inspection product.\n• Oh, I just got the item16 drawing, and it says it's due on May 24th, 2021. I need to do 383 of them, and it costs 16,229 won, and it's an inspection item.\n• There, I need to do 19 drawings for item96. It's until May 30th, 2021. It's 8,333 won. And nothing special.\n• Hey, look quickly. item91 4 I have to do it by June 3rd, 2021. What should I do? The unit price is 36,533 won and there's nothing special about it.\n• item87, right? Doing 196? You can do it until June 18th, 2021, and the unit price is 45,500 won. Oh, and the inspection product"

In [113]:
dataset_for_order[0]

,time,item,cost,urgent,qty
0,date133,item15,25870,1,318
1,date144,item16,16229,1,383
2,date150,item96,8333,0,19
3,date154,item91,36533,0,4
4,date169,item87,45500,1,196


In [114]:
dataset_for_ga[0]

,time,item,cost,urgent,qty,machine,capacity
0,date133,item15,25870,1,318,machine04,12.84
1,date133,item15,25870,1,318,machine05,10.42
2,date133,item15,25870,1,318,machine07,8.69
3,date133,item15,25870,1,318,machine08,9.12
4,date133,item15,25870,1,318,machine09,10.33
...,...,...,...,...,...,...,...
58,date169,item87,45500,1,196,machine19,10.00
59,date169,item87,45500,1,196,machine20,11.37
60,date169,item87,45500,1,196,machine21,7.85
61,date169,item87,45500,1,196,machine32,5.49


In [115]:
# order machine join 정보 묶기
def combine_order_machine_join(data):
    combined_data = list()
    for _, row in data.iterrows():
        combined_data.append(f"• {row['time']} {row['item']} {row['cost']} {row['urgent']} {row['qty']} {row['machine']} {row['capacity']}")

    combined_data = "\n".join(combined_data)
        
    return combined_data

combine_order_machine_join(dataset_for_ga[0])

'• date133 item15 25870 1 318 machine04 12.84\n• date133 item15 25870 1 318 machine05 10.42\n• date133 item15 25870 1 318 machine07 8.69\n• date133 item15 25870 1 318 machine08 9.12\n• date133 item15 25870 1 318 machine09 10.33\n• date133 item15 25870 1 318 machine10 8.99\n• date133 item15 25870 1 318 machine12 12.32\n• date133 item15 25870 1 318 machine14 11.23\n• date133 item15 25870 1 318 machine20 4.0\n• date133 item15 25870 1 318 machine21 9.87\n• date133 item15 25870 1 318 machine23 11.77\n• date133 item15 25870 1 318 machine28 9.54\n• date133 item15 25870 1 318 machine29 13.05\n• date133 item15 25870 1 318 machine30 9.0\n• date133 item15 25870 1 318 machine31 9.0\n• date133 item15 25870 1 318 machine33 4.4\n• date133 item15 25870 1 318 machine34 12.47\n• date133 item15 25870 1 318 machine35 9.4\n• date144 item16 16229 1 383 machine07 4.85\n• date144 item16 16229 1 383 machine08 3.83\n• date144 item16 16229 1 383 machine09 5.37\n• date144 item16 16229 1 383 machine10 3.09\n• date

In [116]:
order_machine_join = [combine_order_machine_join(data) for data in dataset_for_ga]
print(order_machine_join[0])

• date133 item15 25870 1 318 machine04 12.84
• date133 item15 25870 1 318 machine05 10.42
• date133 item15 25870 1 318 machine07 8.69
• date133 item15 25870 1 318 machine08 9.12
• date133 item15 25870 1 318 machine09 10.33
• date133 item15 25870 1 318 machine10 8.99
• date133 item15 25870 1 318 machine12 12.32
• date133 item15 25870 1 318 machine14 11.23
• date133 item15 25870 1 318 machine20 4.0
• date133 item15 25870 1 318 machine21 9.87
• date133 item15 25870 1 318 machine23 11.77
• date133 item15 25870 1 318 machine28 9.54
• date133 item15 25870 1 318 machine29 13.05
• date133 item15 25870 1 318 machine30 9.0
• date133 item15 25870 1 318 machine31 9.0
• date133 item15 25870 1 318 machine33 4.4
• date133 item15 25870 1 318 machine34 12.47
• date133 item15 25870 1 318 machine35 9.4
• date144 item16 16229 1 383 machine07 4.85
• date144 item16 16229 1 383 machine08 3.83
• date144 item16 16229 1 383 machine09 5.37
• date144 item16 16229 1 383 machine10 3.09
• date144 item16 16229 1 383 

In [117]:
def combine_order(data):
    combined_data = list()
    for _, row in data.iterrows():
        combined_data.append(f"• {row['time']} {row['item']} {row['cost']} {row['urgent']} {row['qty']}")

    combined_data = "\n".join(combined_data)
        
    return combined_data

print(combine_order(dataset_for_order[0]))

• date133 item15 25870 1 318
• date144 item16 16229 1 383
• date150 item96 8333 0 19
• date154 item91 36533 0 4
• date169 item87 45500 1 196


In [118]:
order_for_dataset = [combine_order(data) for data in dataset_for_ga]
order_for_dataset[0]

'• date133 item15 25870 1 318\n• date133 item15 25870 1 318\n• date133 item15 25870 1 318\n• date133 item15 25870 1 318\n• date133 item15 25870 1 318\n• date133 item15 25870 1 318\n• date133 item15 25870 1 318\n• date133 item15 25870 1 318\n• date133 item15 25870 1 318\n• date133 item15 25870 1 318\n• date133 item15 25870 1 318\n• date133 item15 25870 1 318\n• date133 item15 25870 1 318\n• date133 item15 25870 1 318\n• date133 item15 25870 1 318\n• date133 item15 25870 1 318\n• date133 item15 25870 1 318\n• date133 item15 25870 1 318\n• date144 item16 16229 1 383\n• date144 item16 16229 1 383\n• date144 item16 16229 1 383\n• date144 item16 16229 1 383\n• date144 item16 16229 1 383\n• date144 item16 16229 1 383\n• date144 item16 16229 1 383\n• date144 item16 16229 1 383\n• date144 item16 16229 1 383\n• date144 item16 16229 1 383\n• date144 item16 16229 1 383\n• date144 item16 16229 1 383\n• date144 item16 16229 1 383\n• date150 item96 8333 0 19\n• date150 item96 8333 0 19\n• date150 ite

In [119]:
def combine_machine(data):
    combined_data = list()
    for _, row in data.iterrows():
        combined_data.append(f"• {row['item']} {row['machine']} {row['capacity']}")

    combined_data = list(set(combined_data))
    combined_data = "\n".join(combined_data)
        
    return combined_data

print(combine_machine(dataset_for_ga[0]))

• item15 machine14 11.23
• item16 machine09 5.37
• item16 machine29 3.86
• item16 machine23 6.25
• item96 machine29 7.12
• item87 machine17 12.0
• item15 machine12 12.32
• item16 machine31 4.53
• item96 machine09 5.92
• item16 machine30 3.8
• item91 machine28 4.33
• item96 machine08 4.89
• item15 machine21 9.87
• item96 machine21 4.6
• item16 machine07 4.85
• item87 machine33 6.9
• item87 machine19 10.0
• item15 machine30 9.0
• item96 machine22 4.54
• item91 machine10 5.32
• item15 machine35 9.4
• item15 machine09 10.33
• item15 machine10 8.99
• item96 machine14 1.03
• item91 machine29 5.32
• item87 machine03 6.5
• item91 machine09 4.03
• item15 machine28 9.54
• item91 machine34 5.25
• item15 machine07 8.69
• item91 machine21 4.8
• item96 machine35 1.83
• item15 machine05 10.42
• item15 machine33 4.4
• item16 machine14 2.73
• item16 machine10 3.09
• item16 machine08 3.83
• item96 machine12 3.5
• item91 machine35 4.5
• item91 machine14 4.0
• item15 machine29 13.05
• item15 machine08 9.1

In [120]:
machine_for_dataset = [combine_machine(data) for data in dataset_for_ga]
machine_for_dataset[0]

'• item15 machine14 11.23\n• item16 machine09 5.37\n• item16 machine29 3.86\n• item16 machine23 6.25\n• item96 machine29 7.12\n• item87 machine17 12.0\n• item15 machine12 12.32\n• item16 machine31 4.53\n• item96 machine09 5.92\n• item16 machine30 3.8\n• item91 machine28 4.33\n• item96 machine08 4.89\n• item15 machine21 9.87\n• item96 machine21 4.6\n• item16 machine07 4.85\n• item87 machine33 6.9\n• item87 machine19 10.0\n• item15 machine30 9.0\n• item96 machine22 4.54\n• item91 machine10 5.32\n• item15 machine35 9.4\n• item15 machine09 10.33\n• item15 machine10 8.99\n• item96 machine14 1.03\n• item91 machine29 5.32\n• item87 machine03 6.5\n• item91 machine09 4.03\n• item15 machine28 9.54\n• item91 machine34 5.25\n• item15 machine07 8.69\n• item91 machine21 4.8\n• item96 machine35 1.83\n• item15 machine05 10.42\n• item15 machine33 4.4\n• item16 machine14 2.73\n• item16 machine10 3.09\n• item16 machine08 3.83\n• item96 machine12 3.5\n• item91 machine35 4.5\n• item91 machine14 4.0\n• item

In [121]:
# gt 정보 텍스트화
def combine_gt(data):
    combined_data = list()
    for _, row in data.iterrows():
        combined_data.append(f"• {row['item']} {row['machine']} {row['time']} {row['qty']}")

    combined_data = "\n".join(combined_data)
        
    return combined_data

print(combine_gt(gt_ls[0]))

• item15 machine04 date133 35
• item15 machine07 date133 47
• item15 machine10 date133 45
• item15 machine14 date133 17
• item15 machine30 date133 9
• item15 machine33 date133 131
• item16 machine10 date144 113
• item16 machine14 date144 196
• item16 machine21 date144 18
• item16 machine28 date144 39
• item96 machine08 date150 9
• item96 machine09 date150 4
• item96 machine10 date150 1
• item96 machine12 date150 8
• item96 machine14 date150 11
• item96 machine21 date150 8
• item96 machine22 date150 18
• item96 machine23 date150 5
• item96 machine28 date150 5
• item96 machine29 date150 5
• item96 machine30 date150 11
• item96 machine34 date150 7
• item96 machine35 date150 4
• item91 machine08 date154 2
• item91 machine10 date154 1
• item91 machine14 date154 1
• item91 machine28 date154 3
• item91 machine29 date154 2
• item91 machine34 date154 4
• item91 machine35 date154 1
• item87 machine03 date169 83
• item87 machine21 date169 16
• item87 machine32 date169 54
• item87 machine33 date16

In [122]:
gt_for_dataset = [combine_gt(data) for data in gt_ls]
gt_for_dataset[0]

'• item15 machine04 date133 35\n• item15 machine07 date133 47\n• item15 machine10 date133 45\n• item15 machine14 date133 17\n• item15 machine30 date133 9\n• item15 machine33 date133 131\n• item16 machine10 date144 113\n• item16 machine14 date144 196\n• item16 machine21 date144 18\n• item16 machine28 date144 39\n• item96 machine08 date150 9\n• item96 machine09 date150 4\n• item96 machine10 date150 1\n• item96 machine12 date150 8\n• item96 machine14 date150 11\n• item96 machine21 date150 8\n• item96 machine22 date150 18\n• item96 machine23 date150 5\n• item96 machine28 date150 5\n• item96 machine29 date150 5\n• item96 machine30 date150 11\n• item96 machine34 date150 7\n• item96 machine35 date150 4\n• item91 machine08 date154 2\n• item91 machine10 date154 1\n• item91 machine14 date154 1\n• item91 machine28 date154 3\n• item91 machine29 date154 2\n• item91 machine34 date154 4\n• item91 machine35 date154 1\n• item87 machine03 date169 83\n• item87 machine21 date169 16\n• item87 machine32 dat

In [123]:
dataset = pd.DataFrame(columns=['order', 'nl', 'machine', 'gt', 'objective'])
dataset

,order,nl,machine,gt,objective


In [124]:
dataset['order'] = order_for_dataset
dataset['nl'] = order_nl_for_dataset
dataset['machine'] = machine_for_dataset
dataset['gt'] = gt_for_dataset
dataset['objective'] = score_ls
dataset.head()

,order,nl,machine,gt,objective
0,• date133 item15 25870 1 318\n• date133 item15...,"• item15, right? 318? You can do it by May 13t...",• item15 machine14 11.23\n• item16 machine09 5...,• item15 machine04 date133 35\n• item15 machin...,1792473
1,• date144 item16 16229 1 383\n• date144 item16...,"• Oh, I just got the item16 drawing, and it sa...",• item16 machine09 5.37\n• item20 machine14 8....,• item16 machine07 date144 1\n• item16 machine...,1532972
2,• date041 item05 2990 0 500\n• date150 item96 ...,"• There, I need to do 19 drawings for item96. ...",• item20 machine14 8.65\n• item96 machine29 7....,• item05 machine10 date041 102\n• item96 machi...,1865500
3,• date041 item05 2990 0 500\n• date049 item95 ...,"• Hey, look quickly. item91 4 I have to do it ...",• item20 machine14 8.65\n• item95 machine19 20...,• item95 machine19 date049 2\n• item91 machine...,1231564
4,• date041 item05 2990 0 500\n• date049 item95 ...,"• item87, right? Doing 196? You can do it unti...",• item20 machine14 8.65\n• item95 machine19 20...,• item05 machine10 date041 36\n• item05 machin...,1037423


In [125]:
dataset['nl_machine'] = (
    "Order information\n" + dataset['nl'] + "\n\n" +
    "Machine information\n" + dataset['machine']
)

dataset['nl_machine_gt'] = (
    "Order information\n" + dataset['nl'] + "\n\n" +
    "Machine information\n" + dataset['machine'] + "\n\n" +
    "Schedule result\n" + dataset['gt']
)

dataset['order_machine'] = (
    "Order information\n" + dataset['order'] + "\n\n" +
    "Machine information\n" + dataset['machine']
)

dataset['order_machine_gt'] = (
    "Order information\n" + dataset['order'] + "\n\n" +
    "Machine information\n" + dataset['machine'] + "\n\n" +
    "Schedule result\n" + dataset['gt']
)

dataset['order_machine_join'] = order_machine_join

dataset['order_machine_join_gt'] = (
    "Order and machine information\n" + dataset['order_machine_join'] + "\n\n" +
    "Schedule result\n" + dataset['gt']
)

dataset.head()

,order,nl,machine,gt,objective,nl_machine,nl_machine_gt,order_machine,order_machine_gt,order_machine_join,order_machine_join_gt
0,• date133 item15 25870 1 318\n• date133 item15...,"• item15, right? 318? You can do it by May 13t...",• item15 machine14 11.23\n• item16 machine09 5...,• item15 machine04 date133 35\n• item15 machin...,1792473,"Order information\n• item15, right? 318? You c...","Order information\n• item15, right? 318? You c...",Order information\n• date133 item15 25870 1 31...,Order information\n• date133 item15 25870 1 31...,• date133 item15 25870 1 318 machine04 12.84\n...,Order and machine information\n• date133 item1...
1,• date144 item16 16229 1 383\n• date144 item16...,"• Oh, I just got the item16 drawing, and it sa...",• item16 machine09 5.37\n• item20 machine14 8....,• item16 machine07 date144 1\n• item16 machine...,1532972,"Order information\n• Oh, I just got the item16...","Order information\n• Oh, I just got the item16...",Order information\n• date144 item16 16229 1 38...,Order information\n• date144 item16 16229 1 38...,• date144 item16 16229 1 383 machine07 4.85\n•...,Order and machine information\n• date144 item1...
2,• date041 item05 2990 0 500\n• date150 item96 ...,"• There, I need to do 19 drawings for item96. ...",• item20 machine14 8.65\n• item96 machine29 7....,• item05 machine10 date041 102\n• item96 machi...,1865500,"Order information\n• There, I need to do 19 dr...","Order information\n• There, I need to do 19 dr...",Order information\n• date041 item05 2990 0 500...,Order information\n• date041 item05 2990 0 500...,• date041 item05 2990 0 500 machine10 4.6\n• d...,Order and machine information\n• date041 item0...
3,• date041 item05 2990 0 500\n• date049 item95 ...,"• Hey, look quickly. item91 4 I have to do it ...",• item20 machine14 8.65\n• item95 machine19 20...,• item95 machine19 date049 2\n• item91 machine...,1231564,"Order information\n• Hey, look quickly. item91...","Order information\n• Hey, look quickly. item91...",Order information\n• date041 item05 2990 0 500...,Order information\n• date041 item05 2990 0 500...,• date041 item05 2990 0 500 machine10 4.6\n• d...,Order and machine information\n• date041 item0...
4,• date041 item05 2990 0 500\n• date049 item95 ...,"• item87, right? Doing 196? You can do it unti...",• item20 machine14 8.65\n• item95 machine19 20...,• item05 machine10 date041 36\n• item05 machin...,1037423,"Order information\n• item87, right? Doing 196?...","Order information\n• item87, right? Doing 196?...",Order information\n• date041 item05 2990 0 500...,Order information\n• date041 item05 2990 0 500...,• date041 item05 2990 0 500 machine10 4.6\n• d...,Order and machine information\n• date041 item0...


In [126]:
print(dataset.iloc[0, -3])

Order information
• date133 item15 25870 1 318
• date133 item15 25870 1 318
• date133 item15 25870 1 318
• date133 item15 25870 1 318
• date133 item15 25870 1 318
• date133 item15 25870 1 318
• date133 item15 25870 1 318
• date133 item15 25870 1 318
• date133 item15 25870 1 318
• date133 item15 25870 1 318
• date133 item15 25870 1 318
• date133 item15 25870 1 318
• date133 item15 25870 1 318
• date133 item15 25870 1 318
• date133 item15 25870 1 318
• date133 item15 25870 1 318
• date133 item15 25870 1 318
• date133 item15 25870 1 318
• date144 item16 16229 1 383
• date144 item16 16229 1 383
• date144 item16 16229 1 383
• date144 item16 16229 1 383
• date144 item16 16229 1 383
• date144 item16 16229 1 383
• date144 item16 16229 1 383
• date144 item16 16229 1 383
• date144 item16 16229 1 383
• date144 item16 16229 1 383
• date144 item16 16229 1 383
• date144 item16 16229 1 383
• date144 item16 16229 1 383
• date150 item96 8333 0 19
• date150 item96 8333 0 19
• date150 item96 8333 0 19
• 

In [127]:
dataset.to_csv('./dataset_n5_20250526.csv')